In [1]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import psycopg2 # needed to get database exception errors when uploading dataframe
import requests # package for getting data from the web
from zipfile import * # package for unzipping zip files


In [2]:

from sql_functions import get_engine

In [3]:
# Specifies path for saving file
path ='data/' 
# Create the data folder
!mkdir {path}

In [4]:
def download_data(year, month):
    # Get the file from the website https://transtats.bts.gov
    zip_file = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip'
    url = (f'https://transtats.bts.gov/PREZIP/{zip_file}')
    # Download the database
    r = requests.get(f'{url}', verify=False)
    # Save database to local file storage
    with open(path+zip_file, 'wb') as f:
        f.write(r.content)
        print(f'--> zip_file with name: {zip_file} downloaded succesfully.' )
        # function to extract the csv files inside the zip files

def extract_zip(year, month):
    # Get the file from the website https://transtats.bts.gov
    zip_file = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip'
    with ZipFile(path+zip_file, 'r') as zip_ref:
        zip_ref.extractall(path)
        csv_file =  zip_ref.namelist()[0]
        print(f'--> zip_file was succesfully extracted to: {csv_file}.' )
    

In [7]:
download_data('2020',8)

/opt/homebrew/Caskroom/miniconda/base/envs/nf_sql/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'transtats.bts.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


--> zip_file with name: On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2020_8.zip downloaded succesfully.


In [8]:
extract_zip('2020',8)

--> zip_file was succesfully extracted to: On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2020_8.csv.


In [10]:
flights = pd.read_csv(path+'Flight_Table_August_2020.csv', low_memory=False)

In [11]:
flights_clean = flights.copy()

In [12]:
flights_clean.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Reporting_Airline', 'DOT_ID_Reporting_Airline',
       'IATA_CODE_Reporting_Airline', 'Tail_Number',
       ...
       'Div4TailNum', 'Div5Airport', 'Div5AirportID', 'Div5AirportSeqID',
       'Div5WheelsOn', 'Div5TotalGTime', 'Div5LongestGTime', 'Div5WheelsOff',
       'Div5TailNum', 'Unnamed: 109'],
      dtype='object', length=110)

In [ ]:
"Year","Month","DayofMonth","FlightDate","Reporting_Airline","DOT_ID_Reporting_Airline","IATA_CODE_Reporting_Airline","Tail_Number","Flight_Number_Reporting_Airline","OriginAirportID","OriginAirportSeqID","OriginCityMarketID","Origin","OriginCityName","OriginState","OriginStateFips","OriginStateName","OriginWac","DestAirportID","DestAirportSeqID","DestCityMarketID","Dest","DestCityName","DestState","DestStateFips","DestStateName","DestWac","CRSDepTime","DepTime","DepDelay","DepDelayMinutes","DepDel15","DepartureDelayGroups","DepTimeBlk","TaxiOut","WheelsOff","WheelsOn","TaxiIn","CRSArrTime","ArrTime","ArrDelay","ArrDelayMinutes","ArrDel15","ArrivalDelayGroups","ArrTimeBlk","Cancelled","CancellationCode","Diverted","CRSElapsedTime","ActualElapsedTime","AirTime","Flights","Distance","DistanceGroup","CarrierDelay","WeatherDelay","NASDelay","SecurityDelay","LateAircraftDelay","FirstDepTime","TotalAddGTime","LongestAddGTime","DivAirportLandings","DivReachedDest","DivActualElapsedTime","DivArrDelay","DivDistance","Div1Airport","Div1AirportID","Div1AirportSeqID","Div1WheelsOn","Div1TotalGTime","Div1LongestGTime","Div1WheelsOff","Div1TailNum","Div2Airport","Div2AirportID","Div2AirportSeqID","Div2WheelsOn","Div2TotalGTime","Div2LongestGTime","Div2WheelsOff","Div2TailNum","Div3Airport","Div3AirportID","Div3AirportSeqID","Div3WheelsOn","Div3TotalGTime","Div3LongestGTime","Div3WheelsOff","Div3TailNum","Div4Airport","Div4AirportID","Div4AirportSeqID","Div4WheelsOn","Div4TotalGTime","Div4LongestGTime","Div4WheelsOff","Div4TailNum","Div5Airport","Div5AirportID","Div5AirportSeqID","Div5WheelsOn","Div5TotalGTime","Div5LongestGTime","Div5WheelsOff","Div5TailNum",

In [15]:
flights.columns.value_counts()

Year                1
Div1Airport         1
Div2WheelsOn        1
Div2AirportSeqID    1
Div2AirportID       1
                   ..
DepDel15            1
DepDelayMinutes     1
DepDelay            1
DepTime             1
Unnamed: 109        1
Name: count, Length: 110, dtype: int64

In [62]:
flights_clean.iloc[:2,20:40]

,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn
0,10397,1039707,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,753,749.0,-4.0,0.0,0.0,-1.0,0700-0759,16.0,805.0,842.0,7.0
1,11423,1142307,31423,DSM,"Des Moines, IA",IA,19,Iowa,61,1424,1433.0,9.0,9.0,0.0,0.0,1400-1459,14.0,1447.0,1503.0,3.0


In [64]:
column_indexes_series = pd.Series(flights_clean.columns)
column_indexes_series[40:50]

40            CRSArrTime
41               ArrTime
42              ArrDelay
43       ArrDelayMinutes
44              ArrDel15
45    ArrivalDelayGroups
46            ArrTimeBlk
47             Cancelled
48      CancellationCode
49              Diverted
dtype: object

In [65]:
wanted_ci_lst =[5,9,14,15,18,23,24,27,29,30,31,40,41,42,47,48,49,51,52,54,56,57,58,59,60,6,10,]

#flights_short = flights_clean.iloc(wanted_ci_lst)
flights_short = flights_clean.iloc[:,wanted_ci_lst]


flights_short.columns = flights_short.columns.str.lower()
flights_short.head()

,flightdate,tail_number,origin,origincityname,originstatename,dest,destcityname,deststatename,crsdeptime,deptime,...,actualelapsedtime,airtime,distance,carrierdelay,weatherdelay,nasdelay,securitydelay,lateaircraftdelay,reporting_airline,flight_number_reporting_airline
0,2020-08-01,N582CA,CAE,"Columbia, SC",South Carolina,ATL,"Atlanta, GA",Georgia,753,749.0,...,60.0,37.0,192.0,NaN,NaN,NaN,NaN,NaN,9E,4628
1,2020-08-01,N915XJ,DTW,"Detroit, MI",Michigan,DSM,"Des Moines, IA",Iowa,1424,1433.0,...,93.0,76.0,534.0,NaN,NaN,NaN,NaN,NaN,9E,4629
2,2020-08-02,N314PQ,DTW,"Detroit, MI",Michigan,DSM,"Des Moines, IA",Iowa,1424,1500.0,...,101.0,80.0,534.0,31.0,0.0,0.0,0.0,0.0,9E,4629
3,2020-08-03,N335PQ,DTW,"Detroit, MI",Michigan,DSM,"Des Moines, IA",Iowa,1424,1419.0,...,113.0,79.0,534.0,NaN,NaN,NaN,NaN,NaN,9E,4629
4,2020-08-04,N319PQ,DTW,"Detroit, MI",Michigan,DSM,"Des Moines, IA",Iowa,1424,1417.0,...,85.0,72.0,534.0,NaN,NaN,NaN,NaN,NaN,9E,4629


In [58]:
flights_short.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376715 entries, 0 to 376714
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   flightdate         376715 non-null  object 
 1   tail_number        375539 non-null  object 
 2   origin             376715 non-null  object 
 3   origincityname     376715 non-null  object 
 4   originstatename    376715 non-null  object 
 5   dest               376715 non-null  object 
 6   destcityname       376715 non-null  object 
 7   deststatename      376715 non-null  object 
 8   deptime            372866 non-null  float64
 9   depdelay           372865 non-null  float64
 10  arrtime            372770 non-null  float64
 11  arrdelay           371969 non-null  float64
 12  cancelled          376715 non-null  float64
 13  cancellationcode   3921 non-null    object 
 14  diverted           376715 non-null  float64
 15  actualelapsedtime  371969 non-null  float64
 16  ai

In [67]:
columns_to_keep = [
                'FlightDate',
                'DepTime',
                'CRSDepTime',
                'DepDelay',
                'ArrTime',
                'CRSArrTime',
                'ArrDelay',
                'Reporting_Airline',
                'Tail_Number',
                'Flight_Number_Reporting_Airline',
                'Origin',
                'Dest',
                'AirTime',
                'ActualElapsedTime',
                'Distance',
                'Cancelled',
                'Diverted'
]

In [66]:

schema = 'hh_analytics_23_4' # UPDATE 'TABLE_SCHEMA' based on schema used in class 
engine = get_engine() # assign engine to be able to query against the database

table_name_sql = f'''SELECT COLUMN_NAME 
                    FROM INFORMATION_SCHEMA.COLUMNS 
                    WHERE TABLE_NAME = 'flights'
                    AND TABLE_SCHEMA ='{schema}'
                    ORDER BY ordinal_position'''
c_names = engine.execute(table_name_sql).fetchall()
c_names

/var/folders/9j/5rxmqbbd545bhw9vljt6mdp40000gn/T/ipykernel_52682/1257290494.py:9: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  c_names = engine.execute(table_name_sql).fetchall()


[('flight_date',),
 ('dep_time',),
 ('sched_dep_time',),
 ('dep_delay',),
 ('arr_time',),
 ('sched_arr_time',),
 ('arr_delay',),
 ('airline',),
 ('tail_number',),
 ('flight_number',),
 ('origin',),
 ('dest',),
 ('air_time',),
 ('actual_elapsed_time',),
 ('distance',),
 ('cancelled',),
 ('diverted',)]

In [68]:
new_column_names=[]
for name in c_names:
    new_column_names.append(name[0])
new_column_names  

['flight_date',
 'dep_time',
 'sched_dep_time',
 'dep_delay',
 'arr_time',
 'sched_arr_time',
 'arr_delay',
 'airline',
 'tail_number',
 'flight_number',
 'origin',
 'dest',
 'air_time',
 'actual_elapsed_time',
 'distance',
 'cancelled',
 'diverted']

In [69]:
def clean_airline_df(df):
    '''
    Transforms a df made from BTS csv file into a df that is ready to be uploaded to SQL
    Set rows=0 for no filtering
    '''

    # Build dataframe including only the columns you want to keep
    df_airline = df.loc[:,columns_to_keep]
     
    # Clean data types and NULLs
    df_airline['FlightDate']= pd.to_datetime(df_airline['FlightDate'], yearfirst=True)
    df_airline['CRSArrTime']= pd.to_numeric(df_airline['CRSArrTime'], downcast='integer', errors='coerce')
    df_airline['Cancelled']= pd.to_numeric(df_airline['Cancelled'], downcast='integer')
    df_airline['Diverted']= pd.to_numeric(df_airline['Diverted'], downcast='integer')
    df_airline['ActualElapsedTime']= pd.to_numeric(df_airline['ActualElapsedTime'], downcast='integer', errors='coerce')
    
    # Rename columns
    df_airline.columns = new_column_names
    
    return df_airline

In [70]:
# Call function and check resulting dataframe
df_clean = clean_airline_df(flights)
df_clean.head()

,flight_date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,airline,tail_number,flight_number,origin,dest,air_time,actual_elapsed_time,distance,cancelled,diverted
0,2020-08-01,749.0,753,-4.0,849.0,905,-16.0,9E,N582CA,4628,CAE,ATL,37.0,60.0,192.0,0,0
1,2020-08-01,1433.0,1424,9.0,1506.0,1510,-4.0,9E,N915XJ,4629,DTW,DSM,76.0,93.0,534.0,0,0
2,2020-08-02,1500.0,1424,36.0,1541.0,1510,31.0,9E,N314PQ,4629,DTW,DSM,80.0,101.0,534.0,0,0
3,2020-08-03,1419.0,1424,-5.0,1512.0,1510,2.0,9E,N335PQ,4629,DTW,DSM,79.0,113.0,534.0,0,0
4,2020-08-04,1417.0,1424,-7.0,1442.0,1510,-28.0,9E,N319PQ,4629,DTW,DSM,72.0,85.0,534.0,0,0


In [84]:
flights_ex = df_clean.copy()
flights_ex[['carrier_delay','weather_delay','nas_delay','security_delay','lateaircraft_delay']] = flights.iloc[:,[56,57,58,59,60]]
flights_ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376715 entries, 0 to 376714
Data columns (total 22 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   flight_date          376715 non-null  datetime64[ns]
 1   dep_time             372866 non-null  float64       
 2   sched_dep_time       376715 non-null  int64         
 3   dep_delay            372865 non-null  float64       
 4   arr_time             372770 non-null  float64       
 5   sched_arr_time       376715 non-null  int16         
 6   arr_delay            371969 non-null  float64       
 7   airline              376715 non-null  object        
 8   tail_number          375539 non-null  object        
 9   flight_number        376715 non-null  int64         
 10  origin               376715 non-null  object        
 11  dest                 376715 non-null  object        
 12  air_time             371969 non-null  float64       
 13  actual_elapsed

In [79]:
def select_airport(df, airports):
    ''' Helper function for filtering the airline dataframe for a subset of airports'''
    df_out = df.loc[(df.origin.isin(airports)) | (df.dest.isin(airports))]
    return df_out

In [83]:
airports=['MKE', 'ORD','MDW', 'IND', 'OMA', 'DSM', 'STL']
if len(airports) > 0:
    flights_MW_group04 = select_airport(flights_ex, airports)
else:
    flights_MW_group04 = df_clean
    
flights_MW_group04.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62568 entries, 1 to 376618
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   flight_date          62568 non-null  datetime64[ns]
 1   dep_time             62067 non-null  float64       
 2   sched_dep_time       62568 non-null  int64         
 3   dep_delay            62067 non-null  float64       
 4   arr_time             62056 non-null  float64       
 5   sched_arr_time       62568 non-null  int16         
 6   arr_delay            61941 non-null  float64       
 7   airline              62568 non-null  object        
 8   tail_number          62455 non-null  object        
 9   flight_number        62568 non-null  int64         
 10  origin               62568 non-null  object        
 11  dest                 62568 non-null  object        
 12  air_time             61941 non-null  float64       
 13  actual_elapsed_time  61941 non-null

In [85]:
# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table, flights_groupname or similar
table_name = 'flights_MW_group04'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        flights_MW_group04.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The flights_MW_group04 table was imported successfully.
